In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('/Users/lionlucky7/Desktop/Coding_Project/data/fbref_total_fielders/total_fielders.csv')

# Features

In [3]:
selected_features = [
    # PLAYER INFO
    "league",
    "season",
    "team",
    "player",
    "pos",
    "age",
    "Playing Time_90s",


    "Standard_Sh/90",
    "Standard_SoT/90",
    "Standard_SoT%",
    "Standard_G/Sh",
    "Standard_G/SoT",
    "Per 90 Minutes_Gls",
    "Per 90 Minutes_Ast",
    "Per 90 Minutes_G+A",
    "Expected_G-xG",
    "Expected_A-xAG",
    "Per 90 Minutes_xG",
    "Per 90 Minutes_xAG",
    "Per 90 Minutes_xG+xAG",

    # PROGRESSION
    "Progression_PrgC",
    "Progression_PrgP",
    "Progression_PrgR",
    "PrgP",
    "Carries_PrgC",

    # PASSING
    "Short_Cmp%",
    "Medium_Cmp%",
    "Long_Cmp%",
    "1/3",
    "PPA",
    "CrsPA",

    # CREATION
    "SCA_SCA90",
    "GCA_GCA90",

    # DEFENDING
    "Tackles_TklW",
    "Challenges_Tkl%",
    "Blocks_Blocks",
    "Performance_Recov",
    "Int",     

    # DRIBBLING
    "Take-Ons_Att",
    "Take-Ons_Succ%",
    "Carries_Mis",

    # RECEIVING
    "Receiving_Rec",
    "Receiving_PrgR",

    # AERIAL
    "Aerial Duels_Won%"
]



columns_nan = [
    "Standard_Sh/90",
    "Standard_SoT/90",
    "Standard_SoT%",
    "Standard_G/Sh",
    "Standard_G/SoT",
    "Per 90 Minutes_Gls",
    "Per 90 Minutes_Ast",
    "Per 90 Minutes_G+A",
    "Expected_G-xG",
    "Expected_A-xAG",
    "Per 90 Minutes_xG",
    "Per 90 Minutes_xAG",
    "Per 90 Minutes_xG+xAG",
    "Progression_PrgC",
    "Progression_PrgP",
    "Progression_PrgR",
    "PrgP",
    "Carries_PrgC",
    "Short_Cmp%",
    "Medium_Cmp%",
    "Long_Cmp%",
    "1/3",
    "PPA",
    "CrsPA",
    "SCA_SCA90",
    "GCA_GCA90",
    "Tackles_TklW",
    "Challenges_Tkl%",
    "Int",
    "Blocks_Blocks",
    "Performance_Recov",
    "Take-Ons_Att",
    "Take-Ons_Succ%",
    "Carries_Mis",
    "Receiving_Rec",
    "Receiving_PrgR",
    "Aerial Duels_Won%"
]

# After

In [4]:
df_train_test = df[(df['season'] >= 1718) & (df['season'] <= 2425)]

# Train

In [5]:
df_train_test = df[(df['season'] >= 1718) & (df['season'] <= 2425)]
df_train_test = df_train_test.select_dtypes(include='number')
df_train_test.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'season', 'born'], inplace=True, errors='ignore')

In [6]:
y_targets = columns_nan
x_features = ['age',
'Playing Time_MP',
'Playing Time_Starts',
'Playing Time_Min',
'Playing Time_90s',
'Performance_Gls',
'Performance_Ast',
'Performance_G+A',
'Performance_G-PK',
'Performance_PK',
'Performance_PKatt',
'Performance_CrdY',
'Performance_CrdR',
'Per 90 Minutes_G-PK',
'Per 90 Minutes_G+A-PK',
'90s',
'Standard_Gls',
'Standard_SoT',
'Standard_PK',
'Standard_PKatt',
'Ast',
'Playing Time_Mn/MP',
'Playing Time_Min%',
'Starts_Starts',
'Starts_Compl',
'Subs_Subs']

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


In [8]:
df_train_test[x_features] = df_train_test[x_features].fillna(df_train_test[x_features].median())
df_train_test = df_train_test.loc[:, ~df_train_test.columns.duplicated()]
y_targets = list(OrderedDict.fromkeys(y_targets))
df_train_test = df_train_test.dropna(subset=y_targets)


X = df_train_test[x_features]
y = df_train_test[y_targets]


X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)

xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    objective='reg:squarederror',
    n_jobs=-1,
    random_state=42
)

multi_model = MultiOutputRegressor(xgb_model)


multi_model.fit(X_train, y_train_scaled)

y_val_pred_scaled = multi_model.predict(X_val)

y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled)

y_val_pred_df = pd.DataFrame(y_val_pred, columns=y_targets, index=y_val.index)


for target in y_targets:
    rmse = np.sqrt(mean_squared_error(y_val[target], y_val_pred_df[target]))
    print(f"{target}: RMSE = {rmse:.2f}")

Standard_Sh/90: RMSE = 0.41
Standard_SoT/90: RMSE = 0.04
Standard_SoT%: RMSE = 12.53
Standard_G/Sh: RMSE = 0.05
Standard_G/SoT: RMSE = 0.00
Per 90 Minutes_Gls: RMSE = 0.01
Per 90 Minutes_Ast: RMSE = 0.01
Per 90 Minutes_G+A: RMSE = 0.01
Expected_G-xG: RMSE = 0.93
Expected_A-xAG: RMSE = 0.87
Per 90 Minutes_xG: RMSE = 0.08
Per 90 Minutes_xAG: RMSE = 0.06
Per 90 Minutes_xG+xAG: RMSE = 0.09
Progression_PrgC: RMSE = 19.57
Progression_PrgP: RMSE = 30.91
Progression_PrgR: RMSE = 38.24
PrgP: RMSE = 30.91
Carries_PrgC: RMSE = 19.57
Short_Cmp%: RMSE = 6.07
Medium_Cmp%: RMSE = 8.86
Long_Cmp%: RMSE = 14.47
1/3: RMSE = 30.31
PPA: RMSE = 8.42
CrsPA: RMSE = 4.35
SCA_SCA90: RMSE = 0.89
GCA_GCA90: RMSE = 0.14
Tackles_TklW: RMSE = 8.18
Challenges_Tkl%: RMSE = 16.18
Int: RMSE = 8.64
Blocks_Blocks: RMSE = 7.19
Performance_Recov: RMSE = 25.63
Take-Ons_Att: RMSE = 19.81
Take-Ons_Succ%: RMSE = 18.19
Carries_Mis: RMSE = 10.70
Receiving_Rec: RMSE = 232.15
Receiving_PrgR: RMSE = 38.24
Aerial Duels_Won%: RMSE = 1

In [9]:
# for i, est in enumerate(multi_model.estimators_):
#     est.save_model(f"/Users/lionlucky7/Desktop/Coding_Project/Models/model_output_{i}.json")

In [10]:
# import joblib
# joblib.dump(multi_model, "/Users/lionlucky7/Desktop/Coding_Project/Models/multioutput_xgb_model.pkl")
# joblib.dump(scaler_y, "/Users/lionlucky7/Desktop/Coding_Project/Models/scaler_y.pkl")

# Application

In [11]:
df_application = df[df['season'].isin([9596, 9697, 9798, 9899, 9900,    1,  102,  203,  304,  405,  506,
        607,  708,  809,  910, 1011, 1112, 1213, 1314, 1415, 1516, 1617])]
df_application.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, errors='ignore')

/var/folders/0_/_3dg0b615p38ptjrygv5db6h0000gn/T/ipykernel_7711/4163594286.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_application.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, errors='ignore')


In [12]:
X_new = df_application[x_features]
X_new[x_features] = X_new[x_features].fillna(X_new[x_features].median())
y_new_pred_scaled = multi_model.predict(X_new)
y_new_pred = scaler_y.inverse_transform(y_new_pred_scaled)

/var/folders/0_/_3dg0b615p38ptjrygv5db6h0000gn/T/ipykernel_7711/2001480899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_new[x_features] = X_new[x_features].fillna(X_new[x_features].median())


In [13]:
y_new_pred_df = pd.DataFrame(y_new_pred, columns=y_targets, index=X_new.index)
y_new_pred_df.tail()

,Standard_Sh/90,Standard_SoT/90,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Per 90 Minutes_Gls,Per 90 Minutes_Ast,Per 90 Minutes_G+A,Expected_G-xG,Expected_A-xAG,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Progression_PrgC,Progression_PrgP,Progression_PrgR,PrgP,Carries_PrgC,Short_Cmp%,Medium_Cmp%,Long_Cmp%,1/3,PPA,CrsPA,SCA_SCA90,GCA_GCA90,Tackles_TklW,Challenges_Tkl%,Int,Blocks_Blocks,Performance_Recov,Take-Ons_Att,Take-Ons_Succ%,Carries_Mis,Receiving_Rec,Receiving_PrgR,Aerial Duels_Won%
50669,1.757528,0.729806,39.665131,0.149095,0.363420,0.250014,0.050892,0.299896,1.626823,-1.024950,0.175394,0.103115,0.272286,44.066521,66.865433,104.841187,66.865433,44.066521,84.395996,83.044228,58.474358,70.445084,14.987369,3.040339,2.288416,0.192989,18.361202,35.026348,22.116365,19.528358,105.232994,45.864182,56.903404,36.162434,805.047974,104.841187,43.010143
50670,0.365166,0.036917,13.861429,0.011790,0.000025,0.000008,0.140627,0.140054,-0.511714,1.289298,0.022531,0.100619,0.114819,70.026657,138.564667,86.517776,138.564667,70.026657,90.234238,81.652428,51.809902,109.140335,20.705372,5.473855,1.646999,0.218415,35.825081,57.278503,33.492710,29.665634,150.210754,20.508474,67.725151,24.041552,1215.569824,86.517776,52.441971
50671,3.496240,1.659412,50.085587,0.152001,0.310163,0.500369,0.078761,0.578170,0.823867,-0.588039,0.404089,0.127545,0.581686,34.768459,40.563980,91.891899,40.563980,34.768459,81.633308,80.221733,59.256744,17.527533,7.663250,1.012750,2.940418,0.354022,5.802560,32.102165,3.671024,8.623730,39.329323,36.861492,46.350914,35.581104,318.980957,91.891899,30.919674
50672,0.393104,0.049807,15.175605,-0.001615,0.000070,-0.000013,0.050100,0.049968,-0.543497,-0.468915,0.026181,0.067369,0.087472,30.699444,86.252205,42.575703,86.252205,30.699444,88.414856,83.322281,58.864830,71.690926,13.829824,5.397426,1.836276,0.147931,24.546999,48.439049,22.136965,21.410717,112.494621,24.855545,55.858547,23.535540,779.023499,42.575703,49.437737
50673,0.639332,0.165340,26.420101,0.009112,0.000015,0.000034,0.169647,0.169903,-0.256498,0.223575,0.044027,0.138329,0.187300,9.990463,27.844135,19.975214,27.844135,9.990463,86.938911,82.084236,55.091232,21.564230,5.036576,1.733368,2.304317,0.209654,7.979522,38.725113,9.715706,7.138992,32.977859,7.085009,50.162403,7.683796,232.913513,19.975214,49.865379


In [14]:
df_application = df_application[selected_features]

In [15]:
df_application.drop(y_targets, axis=1, inplace=True)

In [16]:
df_application_succeed = pd.concat([df_application, y_new_pred_df], axis=1)

In [17]:
df_train = df[(df['season'] >= 1718) & (df['season'] <= 2425)]

In [18]:
df_train = df_train[selected_features]

In [19]:
# df_train = df_train.loc[:, sorted(list(df_train.columns))]
# df_application_succeed = df_application_succeed.loc[:, sorted(list(df_application_succeed.columns))]

In [20]:
pd.concat([df_application_succeed, df_train]).dropna(axis=0).reset_index(drop=True)

,league,season,team,player,pos,age,Playing Time_90s,Standard_Sh/90,Standard_SoT/90,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Per 90 Minutes_Gls,Per 90 Minutes_Ast,Per 90 Minutes_G+A,Expected_G-xG,Expected_A-xAG,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Progression_PrgC,Progression_PrgP,Progression_PrgR,PrgP,Carries_PrgC,Short_Cmp%,Medium_Cmp%,Long_Cmp%,1/3,PPA,CrsPA,SCA_SCA90,GCA_GCA90,Tackles_TklW,Challenges_Tkl%,Int,Blocks_Blocks,Performance_Recov,Take-Ons_Att,Take-Ons_Succ%,Carries_Mis,Receiving_Rec,Receiving_PrgR,Aerial Duels_Won%
0,ENG-Premier League,9596,Arsenal,Adrian Clarke,MF,20.0,4.0,0.857102,0.248008,51.640831,-0.012201,0.000008,0.000025,-4.946168e-07,0.000036,-0.192681,-0.169062,0.044589,0.033763,0.100132,6.672269,11.465883,14.342228,11.465883,6.672269,85.657173,73.956421,54.850430,8.382388,2.385756,0.808830,1.291342,0.151905,4.188464,42.269806,4.017212,3.935005,18.948524,6.322230,65.956444,5.771393,138.537308,14.342228,46.504799
1,ENG-Premier League,9596,Arsenal,Andy Linighan,DF,33.0,17.0,0.711429,0.234110,59.612705,0.008640,-0.000131,-0.000029,6.014853e-02,0.059991,-0.925280,-0.164697,0.054233,0.056836,0.118795,17.869442,64.393860,18.020020,64.393860,17.869442,89.029701,82.263969,53.480415,54.441574,10.621790,2.655607,1.324638,0.119615,15.449721,49.192299,23.400312,19.506527,85.919594,10.280685,72.111046,13.646759,715.460754,18.020020,57.728863
2,ENG-Premier League,9596,Arsenal,David Hillier,MF,25.0,3.2,0.785623,0.302317,80.855850,0.004521,0.000006,0.000119,-7.310947e-06,0.000018,-0.226400,-0.122053,0.081984,0.025877,0.106883,5.744287,8.582197,12.605628,8.582197,5.744287,86.918739,80.937698,60.017151,6.103043,2.017227,0.878647,1.094923,0.089892,2.869823,59.763588,3.474088,3.351405,14.179519,5.175303,46.087265,3.930744,113.189857,12.605628,40.776070
3,ENG-Premier League,9596,Arsenal,David Platt,MF,29.0,27.6,2.042670,0.754766,37.164837,0.108208,0.290733,0.220871,1.076498e-01,0.330301,0.238164,-0.857170,0.231652,0.128782,0.364271,76.522064,133.973877,190.505310,133.973877,76.522064,85.427437,80.604195,54.785179,108.602798,30.180536,4.810087,3.074432,0.282727,25.052784,38.812508,18.696262,27.228153,133.427689,67.286133,59.679337,60.898094,1073.498169,190.505310,41.399044
4,ENG-Premier League,9596,Arsenal,Dennis Bergkamp,FW,26.0,32.5,3.225213,1.247765,41.037598,0.122917,0.275904,0.340652,2.467427e-01,0.578348,0.040998,2.093678,0.295019,0.187944,0.495995,132.113831,113.043396,277.116119,113.043396,132.113831,82.971207,71.615143,56.161469,84.041008,45.889355,10.305880,3.894287,0.475668,17.118351,31.516573,18.695976,26.676235,142.996704,104.146881,55.827858,72.153107,1252.448486,277.116119,50.784229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61814,ITA-Serie A,2425,Venezia,Magnus Kofod Andersen,MF,25.0,7.7,1.040000,0.260000,25.000000,0.130000,0.500000,0.130000,1.300000e-01,0.260000,0.700000,0.500000,0.040000,0.060000,0.100000,7.000000,24.000000,18.000000,24.000000,7.000000,87.400000,90.400000,45.000000,18.000000,3.000000,0.000000,2.590000,0.390000,4.000000,40.000000,6.000000,7.000000,31.000000,4.000000,50.000000,8.000000,238.000000,18.000000,53.300000
61815,ITA-Serie A,2425,Venezia,Marin Šverko,DF,26.0,12.4,0.240000,0.080000,33.300000,0.330000,1.000000,0.080000,0.000000e+00,0.080000,0.700000,-0.300000,0.030000,0.030000,0.050000,8.000000,38.000000,1.000000,38.000000,8.000000,90.700000,85.800000,56.300000,39.000000,5.000000,0.000000,1.050000,0.080000,9.000000,83.300000,16.000000,10.000000,44.000000,10.000000,60.000000,8.000000,388.000000,1.000000,50.000000
61816,ITA-Serie A,2425,Venezia,Michael Svoboda,DF,25.0,14.6,0.270000,0.270000,100.000000,0.250000,0.250000,0.070000,0.000000e+00,0.070000,0.800000,-0.600000,0.010000,0.040000,0.060000,13.000000,43.000000,5.000000,43.000000,13.000000,90.600000,89.400000,51.900000,44.000000,2.000000,0.000000,1.100000,0.000000,14.000000,42